In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Importando bibliotecas necessárias
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing import image
import numpy as np

# Definindo o gerador de dados
datagen = ImageDataGenerator(
    rescale=1./255,         # Normalização
    validation_split=0.2    # Separar 20% para validação
)

# Carregar os dados de treino
train_data = datagen.flow_from_directory(
    '/content/drive/MyDrive/Colab Notebooks/AULA_10/dataset',             # Caminho para o dataset
    target_size=(128, 128), # Redimensionar as imagens
    batch_size=32,          # Número de amostras por batch
    class_mode='categorical', # Classificação multiclasses
    subset='training'       # Conjunto de treinamento
)

# Carregar os dados de validação
val_data = datagen.flow_from_directory(
    '/content/drive/MyDrive/Colab Notebooks/AULA_10/dataset',
    target_size=(128, 128),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

# Construindo o modelo
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    MaxPooling2D(pool_size=(2, 2)),

    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),

    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),

    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),

    Dense(6, activation='softmax')  # 15 classes (uma para cada pessoa)
])

# Compilando o modelo
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Treinando o modelo
history = model.fit(
    train_data,
    epochs=25,
    validation_data=val_data
)

# Salvando a estrutura do modelo
model_structure = model.to_json()
with open('modelo_classificacao_pessoas.json', 'w') as json_file:
    json_file.write(model_structure)

# Salvando os pesos do modelo
model.save_weights('modelo_classificacao_pessoas.weights.h5')

# Salvando os rótulos em um arquivo TXT
labels = train_data.class_indices  # Obter os rótulos do dataset
with open('rotulos.txt', 'w') as txt_file:
    for label, index in labels.items():
        txt_file.write(f"{label}: {index}\n")  # Salva o rótulo e seu índice